In [1]:
push!(LOAD_PATH, "/home/amir/Work/gdmrg/src/")
import Tmp
using Plots
using LaTeXStrings

In [2]:
sz_half = Tmp.sz_half;
sp_half = Tmp.sp_half;
sm_half = Tmp.sm_half;

In [3]:
### loading example files for illustration ###
#include("example-01-compareHeisVsRandom.jl")

In [4]:
reload("Tmp")
include("test/runtests.jl")

Test Summary: | Pass  

Total
MPS stuff     |   18     18
Test Summary: | Pass  Total
apply twosite |    2      2
Test Summary: | Pass  Total
tools         |    7      7
Test Summary: | Pass  Total
ED stuff      |   10     10
Test Summary: | Pass  Total
Fishman       |    5      5


Base.Test.DefaultTestSet("Fishman", Any[Base.Test.DefaultTestSet("general", Any[], 2, false), Base.Test.DefaultTestSet("compare XY with Free Fermions", Any[], 3, false)], 0, false)

In [ ]:
Lx = 8

Hhs = Tmp.xxz(Lx)
eheis, vheis = eigs(Hhs, nev=1, which=:SR)
mpsheis = Tmp.MPS(Lx, 2, vheis[:,1])
measheis_zz = Tmp.measure(mpsheis, [sz_half, sz_half])
measheis_pm = Tmp.measure(mpsheis, [sp_half, sm_half])

corr_matrix = Tmp.correlation_matrix(Tmp.hamiltonian_matrix(Lx, -1.0, 1.), div(Lx,2))
gmps = Tmp.makeGMPS!(corr_matrix, 1.e-8)
mpsgutz = Tmp.gutzwiller_project(Tmp.generateMPS_twoflavors(gmps, 100))
@test Tmp.dims_are_consistent(mpsgutz)
@show gutznorm = Tmp.norm(mpsgutz)[1,1]

measgutz_zz = Tmp.measure(mpsgutz, [sz_half, sz_half])/gutznorm
measgutz_pm = Tmp.measure(mpsgutz, [sp_half, sm_half])/gutznorm
mpo = Tmp.MPO{Float64}(Lx, 2)
@show eheis, Tmp.measure(mpsgutz, mpo)/gutznorm, Tmp.measure(mpsheis, mpo)

p1 = plot(real.(measheis_zz[1:Lx-1]), label="Heisenberg")
plot!(p1, real.(measgutz_zz[1:Lx-1]), label="Gutzwiller-projected fermions")
plot!(p1, imag.(measgutz_zz[1:Lx-1]), label="Gutzwiller-projected fermions imaginary")

p2 = plot(real.(measheis_pm[1:Lx-1]), label="Heisenberg")
plot!(p2, real.(measgutz_pm[1:Lx-1]), label="Gutzwiller-projected fermions")
plot!(p2, imag.(measgutz_pm[1:Lx-1]), label="Gutzwiller-projected fermions imaginary")

println("here")

vneheis = Tmp.entropy(Tmp.entanglements!(mpsheis))

println("here")
@show mpsgutz.dims

vnegutz = Tmp.entropy(Tmp.entanglements!(mpsgutz))

p3 = plot(vneheis)
plot!(p3, vnegutz)

p4 = plot(collect(1:Lx))
plot(p1,p2,p3,p4, layout=(2,2))

gutznorm = (Tmp.norm(mpsgutz))[1, 1] = 0.03300726448939829 + 0.0im
(eheis, Tmp.measure(mpsgutz, mpo) / gutznorm, Tmp.measure(mpsheis, mpo)) = 